In [1]:
!pip install 'litgpt[all]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.7/157.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
!litgpt download microsoft/phi-2

repo_id: microsoft/phi-2
Setting HF_HUB_ENABLE_HF_TRANSFER=1
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1194: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
config.json: 100% 735/735 [00:00<00:00, 5.31MB/s]
generation_config.json: 100% 124/124 [00:00<00:00, 781kB/s]
model-00001-of-00002.safetensors: 100% 5.00G/5.00G [00:16<00:00, 298MB/s]
mode

In [4]:
!litgpt finetune_lora checkpoints/microsoft/phi-2   --data JSON   --data.json_path method_train.json   --data.val_split_fraction 0.1   --out_dir out/phi-2-finetuned   --train.max_seq_length 2048

{'checkpoint_dir': PosixPath('checkpoints/microsoft/phi-2'),
 'data': JSON(json_path=PosixPath('method_train.json'),
              mask_prompt=False,
              val_split_fraction=0.1,
              prompt_style=<litgpt.prompts.Alpaca object at 0x798f572c4d00>,
              ignore_index=-100,
              seed=42,
              num_workers=4),
 'devices': 1,
 'eval': EvalArgs(interval=100,
                  max_new_tokens=100,
                  max_iters=100,
                  initial_validation=False,
                  final_validation=True),
 'logger_name': 'csv',
 'lora_alpha': 16,
 'lora_dropout': 0.05,
 'lora_head': False,
 'lora_key': False,
 'lora_mlp': False,
 'lora_projection': False,
 'lora_query': True,
 'lora_r': 8,
 'lora_value': True,
 'optimizer': 'AdamW',
 'out_dir': PosixPath('out/phi-2-finetuned'),
 'precision': None,
 'quantize': None,
 'seed': 1337,
 'train': TrainArgs(save_interval=1000,
                    log_interval=1,
                    global_batch_size

In [10]:
!litgpt chat out/phi-2-finetuned/final

{'checkpoint_dir': PosixPath('out/phi-2-finetuned/final'),
 'compile': False,
 'multiline': False,
 'precision': None,
 'quantize': None,
 'temperature': 0.8,
 'top_k': 200,
 'top_p': 1.0}
Now chatting with phi-2.
To exit, press 'Enter' on an empty prompt.

Seed set to 1234
>> Prompt: 


In [11]:
!nohup litgpt serve out/phi-2-finetuned/final &

nohup: appending output to 'nohup.out'


In [12]:
import json
import requests
import pandas as pd

# Function to generate a response from the model
def generate_response(prompt):
    try:
        response = requests.post(
            "http://127.0.0.1:8000/predict",
            json={"prompt": prompt}
        )
        response.raise_for_status()  # Check for request errors
        output = response.json().get("output", "")
        print(f"Response: {output}")
        return output
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return ""

def read_json(json_file):
    try:
        # Load JSON data
        with open(json_file, 'r') as f:
            json_data = json.load(f)
        return json_data
    except FileNotFoundError:
        print(f"File not found: {json_file}")
        return []

def main():
    try:
        df = pd.read_excel('method_vaild.xlsx', engine='openpyxl')
        json_data = read_json('method_vaild.json')

        # Check if json_data is not empty
        if not json_data:
            print("No data to process.")
            return

        # Write responses to Excel
        for item in json_data:
            instruction = item['instruction']
            response = generate_response(instruction.strip())

            repo_name = item['meta']['Repo name']
            commit_hash = item['meta']['commit hash']
            file_name = item['meta']['file']
            print(f"Processing '{file_name}':")

            # Find the matching row in the DataFrame
            match = df[(df['Repo name'] == repo_name) & (df['commit hash'] == commit_hash) & (df['before/after file'] == file_name)]
            if not match.empty:
                row_index = match.index[0]
                contains_yes = 'yes' in response
                contains_no = 'no' in response
                exist_str = 'Y' if contains_yes else 'N' if contains_no else 'UN'
                df.loc[row_index, 'has NPE'] = exist_str

        df.to_excel('method_vaild.xlsx', index=False, engine='openpyxl')
        print(f"Updated spreadsheet 'method_vaild.xlsx' with new data.")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


Response: no
Processing 'createFromLookAt_after':
Response: no
Processing 'doCheckAssignedLabelString_before':
Response: no
Processing 'assembleSqlFilter_after':
Response: 
no
Processing 'buildAndSaveNewProposal_after':
Response: yes
Processing 'ServerStarted_after':
Response: yes
Processing 'parseDate_after':
Response: yes
Processing 'main_after':
Response: no
Processing 'updateConv_after':
Response: no
Processing 'readPayload_after':
Response: yes
Processing 'parseDate_before':
Response: 
no
Processing 'callBackReceive_after':
Response: no
Processing 'refreshPlayers_after':
Response: no
Processing 'getLastDeliveryTimeByPartnerId_after':
Response: `yes
Processing 'zPositionHook_after':
Response: yes
Processing 'init_before':
Response: no
Processing 'search_after':
Response: no
Processing 'executeFallbackIfDocumentFailedToVerify_after':
Response: yes
Processing 'createModule_after':
Response: yes
Processing 'create_after':
Response: no
Processing 'updatePoints_after':
Response: yes
Pro